In [ ]:
#| default_exp utils_seo

In [ ]:
#| export

from __future__ import annotations
from typing import List, Dict, Optional
from datetime import datetime
from html import escape

In [ ]:
from nbdev.showdoc import show_doc

## meta tag generator

Generate SEO-optimized `<head>` tags for blog posts.

In [ ]:
#| export

def generate_head_tags(
    title: str, # Page title
    description: str, # Page description (150-160 chars optimal)
    url: str, # Canonical URL
    image_url: Optional[str] = None, # OpenGraph image URL
    article_published: Optional[datetime] = None, # Publication date
    article_modified: Optional[datetime] = None, # Last modified date
    author: Optional[str] = None # Author name
) -> List[tuple]:
    """
    Generate meta tags for SEO.
    
    Returns list of (tag_name, attributes_dict) tuples for FastHTML components.
    Includes standard, OpenGraph, and Twitter Card tags.
    
    Example:
        ```python
        from fasthtml.common import *
        
        tags = generate_head_tags(
            title='My Blog Post',
            description='Learn about Python',
            url='https://example.com/blog/my-post',
            image_url='https://example.com/image.jpg'
        )
        
        # Use in FastHTML app
        @app.get('/blog/my-post')
        def post():
            return Html(
                Head(
                    *[Meta(**attrs) if tag == 'meta' else Link(**attrs) 
                      for tag, attrs in tags]
                ),
                Body('...')
            )
        ```
    """
    tags = []
    
    # Standard meta tags
    tags.append(('title', {'content': title}))
    tags.append(('meta', {'name': 'description', 'content': description}))
    
    # Canonical URL
    tags.append(('link', {'rel': 'canonical', 'href': url}))
    
    # OpenGraph tags (Facebook, WhatsApp, etc.)
    tags.extend([
        ('meta', {'property': 'og:title', 'content': title}),
        ('meta', {'property': 'og:description', 'content': description}),
        ('meta', {'property': 'og:url', 'content': url}),
        ('meta', {'property': 'og:type', 'content': 'article'})
    ])
    
    if image_url:
        tags.append(('meta', {'property': 'og:image', 'content': image_url}))
    
    # Article metadata
    if article_published:
        tags.append(('meta', {
            'property': 'article:published_time',
            'content': article_published.isoformat()
        }))
    
    if article_modified:
        tags.append(('meta', {
            'property': 'article:modified_time',
            'content': article_modified.isoformat()
        }))
    
    if author:
        tags.append(('meta', {'property': 'article:author', 'content': author}))
    
    # Twitter Card tags
    tags.extend([
        ('meta', {'name': 'twitter:card', 'content': 'summary_large_image'}),
        ('meta', {'name': 'twitter:title', 'content': title}),
        ('meta', {'name': 'twitter:description', 'content': description})
    ])
    
    if image_url:
        tags.append(('meta', {'name': 'twitter:image', 'content': image_url}))
    
    return tags

In [ ]:
show_doc(generate_head_tags)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_seo.py#L13){target="_blank" style="float:right; font-size:smaller"}

### generate_head_tags

>      generate_head_tags (title:str, description:str, url:str,
>                          image_url:Optional[str]=None,
>                          article_published:Optional[datetime.datetime]=None,
>                          article_modified:Optional[datetime.datetime]=None,
>                          author:Optional[str]=None)

*Generate meta tags for SEO.*

Returns list of (tag_name, attributes_dict) tuples for FastHTML components.
Includes standard, OpenGraph, and Twitter Card tags.

Example:
    ```python
    from fasthtml.common import *

    tags = generate_head_tags(
        title='My Blog Post',
        description='Learn about Python',
        url='https://example.com/blog/my-post',
        image_url='https://example.com/image.jpg'
    )

    # Use in FastHTML app
    @app.get('/blog/my-post')
    def post():
        return Html(
            Head(
                *[Meta(**attrs) if tag == 'meta' else Link(**attrs) 
                  for tag, attrs in tags]
            ),
            Body('...')
        )
    ```

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| title | str |  | Page title |
| description | str |  | Page description (150-160 chars optimal) |
| url | str |  | Canonical URL |
| image_url | Optional | None | OpenGraph image URL |
| article_published | Optional | None | Publication date |
| article_modified | Optional | None | Last modified date |
| author | Optional | None | Author name |
| **Returns** | **List** |  |  |

## sitemap generator

Generate XML sitemap for search engine crawlers.

In [ ]:
#| export

def generate_sitemap_xml(
    posts: List[Dict], # List of posts from PostLoader
    base_url: str, # Base URL (e.g., 'https://example.com')
    blog_path: str = '/blog' # Blog path prefix
) -> str:
    """
    Generate XML sitemap for blog posts.
    
    Returns sitemap XML string with proper structure and lastmod dates.
    
    Example:
        ```python
        from fasthtml.common import *
        from fh_saas.utils_blog import PostLoader
        
        app = FastHTML()
        loader = PostLoader('blog/posts')
        
        @app.get('/sitemap.xml')
        def sitemap():
            posts = loader.load_posts()
            xml = generate_sitemap_xml(
                posts=posts,
                base_url='https://example.com',
                blog_path='/blog'
            )
            return Response(xml, media_type='application/xml')
        ```
    """
    xml_parts = [
        '<?xml version="1.0" encoding="UTF-8"?>',
        '<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9">'
    ]
    
    # Add blog index
    xml_parts.append(f'  <url>')
    xml_parts.append(f'    <loc>{escape(base_url + blog_path)}</loc>')
    xml_parts.append(f'    <changefreq>daily</changefreq>')
    xml_parts.append(f'    <priority>1.0</priority>')
    xml_parts.append(f'  </url>')
    
    # Add each post
    for post in posts:
        url = f"{base_url}{blog_path}/{post['slug']}"
        xml_parts.append(f'  <url>')
        xml_parts.append(f'    <loc>{escape(url)}</loc>')
        
        if post.get('date'):
            lastmod = post['date'].strftime('%Y-%m-%d')
            xml_parts.append(f'    <lastmod>{lastmod}</lastmod>')
        
        xml_parts.append(f'    <changefreq>monthly</changefreq>')
        xml_parts.append(f'    <priority>0.8</priority>')
        xml_parts.append(f'  </url>')
    
    xml_parts.append('</urlset>')
    return '\n'.join(xml_parts)

In [ ]:
show_doc(generate_sitemap_xml)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_seo.py#L100){target="_blank" style="float:right; font-size:smaller"}

### generate_sitemap_xml

>      generate_sitemap_xml (posts:List[Dict], base_url:str,
>                            blog_path:str='/blog')

*Generate XML sitemap for blog posts.*

Returns sitemap XML string with proper structure and lastmod dates.

Example:
    ```python
    from fasthtml.common import *
    from fh_saas.utils_blog import PostLoader

    app = FastHTML()
    loader = PostLoader('blog/posts')

    @app.get('/sitemap.xml')
    def sitemap():
        posts = loader.load_posts()
        xml = generate_sitemap_xml(
            posts=posts,
            base_url='https://example.com',
            blog_path='/blog'
        )
        return Response(xml, media_type='application/xml')
    ```

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| posts | List |  | List of posts from PostLoader |
| base_url | str |  | Base URL (e.g., 'https://example.com') |
| blog_path | str | /blog | Blog path prefix |
| **Returns** | **str** |  |  |

## rss feed generator

Generate RSS 2.0 feed for blog subscribers.

In [ ]:
#| export

def generate_rss_xml(
    posts: List[Dict], # List of posts from PostLoader
    blog_title: str, # Blog title
    blog_description: str, # Blog description
    base_url: str, # Base URL
    blog_path: str = '/blog' # Blog path prefix
) -> str:
    """
    Generate RSS 2.0 feed for blog posts.
    
    Returns RSS XML string for feed readers (Feedly, etc.).
    
    Example:
        ```python
        from fasthtml.common import *
        from fh_saas.utils_blog import PostLoader
        
        app = FastHTML()
        loader = PostLoader('blog/posts')
        
        @app.get('/rss.xml')
        def rss():
            posts = loader.load_posts()[:20]  # Latest 20 posts
            xml = generate_rss_xml(
                posts=posts,
                blog_title='My Blog',
                blog_description='Tech tutorials and insights',
                base_url='https://example.com'
            )
            return Response(xml, media_type='application/xml')
        ```
    """
    from email.utils import formatdate
    
    xml_parts = [
        '<?xml version="1.0" encoding="UTF-8"?>',
        '<rss version="2.0" xmlns:atom="http://www.w3.org/2005/Atom">',
        '  <channel>',
        f'    <title>{escape(blog_title)}</title>',
        f'    <link>{escape(base_url + blog_path)}</link>',
        f'    <description>{escape(blog_description)}</description>',
        f'    <atom:link href="{escape(base_url)}/rss.xml" rel="self" type="application/rss+xml" />'
    ]
    
    # Add items
    for post in posts:
        url = f"{base_url}{blog_path}/{post['slug']}"
        
        xml_parts.append('    <item>')
        xml_parts.append(f'      <title>{escape(post["title"])}</title>')
        xml_parts.append(f'      <link>{escape(url)}</link>')
        xml_parts.append(f'      <guid>{escape(url)}</guid>')
        
        if post.get('description'):
            xml_parts.append(f'      <description>{escape(post["description"])}</description>')
        
        if post.get('date'):
            pub_date = formatdate(post['date'].timestamp(), usegmt=True)
            xml_parts.append(f'      <pubDate>{pub_date}</pubDate>')
        
        if post.get('author'):
            xml_parts.append(f'      <author>{escape(post["author"])}</author>')
        
        xml_parts.append('    </item>')
    
    xml_parts.append('  </channel>')
    xml_parts.append('</rss>')
    
    return '\n'.join(xml_parts)

In [ ]:
show_doc(generate_rss_xml)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_seo.py#L159){target="_blank" style="float:right; font-size:smaller"}

### generate_rss_xml

>      generate_rss_xml (posts:List[Dict], blog_title:str, blog_description:str,
>                        base_url:str, blog_path:str='/blog')

*Generate RSS 2.0 feed for blog posts.*

Returns RSS XML string for feed readers (Feedly, etc.).

Example:
    ```python
    from fasthtml.common import *
    from fh_saas.utils_blog import PostLoader

    app = FastHTML()
    loader = PostLoader('blog/posts')

    @app.get('/rss.xml')
    def rss():
        posts = loader.load_posts()[:20]  # Latest 20 posts
        xml = generate_rss_xml(
            posts=posts,
            blog_title='My Blog',
            blog_description='Tech tutorials and insights',
            base_url='https://example.com'
        )
        return Response(xml, media_type='application/xml')
    ```

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| posts | List |  | List of posts from PostLoader |
| blog_title | str |  | Blog title |
| blog_description | str |  | Blog description |
| base_url | str |  | Base URL |
| blog_path | str | /blog | Blog path prefix |
| **Returns** | **str** |  |  |

## FastHTML Integration Example

Complete SEO setup with FastHTML:

```python
from fasthtml.common import *
from fh_saas.utils_blog import PostLoader, MarkdownEngine
from fh_saas.utils_seo import generate_head_tags, generate_sitemap_xml, generate_rss_xml

app = FastHTML()
loader = PostLoader('blog/posts')
engine = MarkdownEngine()

@app.get('/blog/{slug}')
def blog_post(slug: str):
    post = loader.get_post(slug)
    if not post:
        return 'Post not found', 404
    
    # Generate SEO tags
    tags = generate_head_tags(
        title=post['title'],
        description=post['description'] or post['body'][:160],
        url=f"https://example.com/blog/{slug}",
        image_url=post.get('image'),
        article_published=post['date'],
        author=post.get('author')
    )
    
    # Render markdown
    html_content = engine.render(post['body'])
    
    return Html(
        Head(
            Title(post['title']),
            *[Meta(**attrs) if tag == 'meta' else Link(**attrs) 
              for tag, attrs in tags]
        ),
        Body(
            Article(NotStr(html_content))
        )
    )

@app.get('/sitemap.xml')
def sitemap():
    posts = loader.load_posts()
    xml = generate_sitemap_xml(posts, 'https://example.com')
    return Response(xml, media_type='application/xml')

@app.get('/rss.xml')
def rss():
    posts = loader.load_posts()[:20]
    xml = generate_rss_xml(
        posts=posts,
        blog_title='My Blog',
        blog_description='Tech tutorials',
        base_url='https://example.com'
    )
    return Response(xml, media_type='application/xml')
```

In [ ]:
#| hide

import nbdev as nb
nb.nbdev_export()